In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hybrid-dataset/Hybrid_trainset.pkl
/kaggle/input/hybrid-dataset/Hybrid_testset.pkl


In [2]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as rSquared
from keras.models import Sequential
from keras.layers import Conv2D, Dense

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Concatenate,Input
from tensorflow.data import Dataset
from tensorflow.keras.models import Model

import cv2
from PIL import Image


2024-07-24 09:40:17.551227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 09:40:17.551336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 09:40:17.684612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
with open('/kaggle/input/hybrid-dataset/Hybrid_trainset.pkl','rb') as f:
    train = pickle.load(f)

with open('/kaggle/input/hybrid-dataset/Hybrid_testset.pkl','rb') as f:
    test = pickle.load(f)

In [5]:

#dataset = list(zip(Scaled_imageset,PM_value,Green_index_lst,Urban_index_lst,Near_road_lst,Season_lst,is_week_lstLSTM_lst))
train

[(array([[[ 53,  68,  53],
          [ 43,  57,  49],
          [ 49,  61,  50],
          ...,
          [205, 145, 127],
          [188, 138, 116],
          [170, 133, 108]],
  
         [[ 60,  79,  55],
          [ 49,  65,  53],
          [ 36,  44,  42],
          ...,
          [195, 147, 127],
          [183, 140, 115],
          [170, 134, 105]],
  
         [[ 67, 101,  60],
          [ 52,  71,  56],
          [ 31,  28,  36],
          ...,
          [186, 158, 130],
          [179, 145, 119],
          [173, 136, 107]],
  
         ...,
  
         [[ 60,  69,  71],
          [ 66,  69,  73],
          [ 66,  68,  76],
          ...,
          [ 80,  80,  64],
          [ 96,  90,  71],
          [112,  97,  84]],
  
         [[ 65,  70,  70],
          [ 63,  69,  75],
          [ 61,  66,  73],
          ...,
          [ 76,  73,  71],
          [ 92,  80,  78],
          [114,  93,  93]],
  
         [[ 63,  70,  73],
          [ 61,  68,  74],
          [ 58,  68,  68

In [45]:
image_size=224


X_train_img = np.array([item[0] for item in train]).reshape(-1,image_size,image_size,3)*(1./255)
X_test_img = np.array([item[0] for item in test]).reshape(-1,image_size,image_size,3)*(1./255)

y_train = np.array([item[1] for item in train]).reshape(-1)
y_test = np.array([item[1] for item in test]).reshape(-1)

X_train_auxiliary1 = np.array([item[2] for item in train]).reshape((len(train)),1)
X_test_auxiliary1= np.array([item[2] for item in test]).reshape((len(test)),1)

X_train_auxiliary2 = np.array([item[3] for item in train]).reshape((len(train)),1)
X_test_auxiliary2= np.array([item[3] for item in test]).reshape((len(test)),1)

X_train_auxiliary3 = np.array([item[4] for item in train]).reshape((len(train)),1)
X_test_auxiliary3= np.array([item[4] for item in test]).reshape((len(test)),1)

X_train_auxiliary4 = np.array([item[5] for item in train]).reshape((len(train)),1)
X_test_auxiliary4= np.array([item[5] for item in test]).reshape((len(test)),1)

#X_train_auxiliary5 = np.array([item[6] for item in train]).reshape((len(train)),1)
#X_test_auxiliary5= np.array([item[6] for item in test]).reshape((len(test)),1)

#X_train_auxiliary6 = np.array([item[7] for item in train]).reshape((len(train)),1)
#X_test_auxiliary6 = np.array([item[7] for item in test]).reshape((len(test)),1)

**Building the Model**

In [46]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Flatten, LSTM, concatenate,MaxPooling2D,Dropout


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [47]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define inputs
image_input=Input(shape=(224,224,3))

# Pass the image input through the base model
x = base_model(image_input)
x = Flatten()(x)

auxiliary_input1=Input(shape=(1,))
auxiliary_input2=Input(shape=(1,))
auxiliary_input3=Input(shape=(1,))
auxiliary_input4=Input(shape=(1,))
#auxiliary_input5=Input(shape=(1,))
#auxiliary_input6=Input(shape=(1,))


#combined_input=concatenate([x,auxiliary_input1,auxiliary_input2,auxiliary_input3,auxiliary_input4,auxiliary_input5,auxiliary_input6])
combined_input=concatenate([x,auxiliary_input1,auxiliary_input2,auxiliary_input3,auxiliary_input4])
x=Dense(128, activation='relu')(combined_input)
x=Dense(64, activation='relu')(x)
z=Dense(1, activation='linear')(x)
model = Model(inputs=[image_input,auxiliary_input1,auxiliary_input2,auxiliary_input3,auxiliary_input4], outputs=z)
#model = Model(inputs=[image_input,auxiliary_input1,auxiliary_input2,auxiliary_input3,auxiliary_input4,auxiliary_input5,auxiliary_input6], outputs=z)


In [48]:
# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=['mae'],run_eagerly=True)
early_stop = EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [49]:
epochsize = 50
#model = model.fit(X_train, y_train, validation_split= 0.1,epochs=10, batch_size = 100, callbacks=[early_stop],verbose= 1,shuffle=False)
history = model.fit(x=[X_train_img,X_train_auxiliary1,X_train_auxiliary2,X_train_auxiliary3,X_train_auxiliary4], y=y_train, validation_split= 0.1,epochs=epochsize, batch_size=100, callbacks=[early_stop],verbose= 1,shuffle=False)
#history = model.fit(x=[X_train_img,X_train_auxiliary1,X_train_auxiliary2,X_train_auxiliary3,X_train_auxiliary4,X_train_auxiliary5,X_train_auxiliary6], y=y_train, validation_split= 0.1,epochs=epochsize, batch_size=100, callbacks=[early_stop],verbose= 1,shuffle=False)

Epoch 1/50


ResourceExhaustedError: Exception encountered when calling Conv2D.call().

[1m{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[100,224,224,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D][0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(100, 224, 224, 3), dtype=float32)

In [ ]:
print("Shape of X_train_img:", X_train_img.shape)
print("Shape of X_train_auxiliary1:", X_train_auxiliary1.shape)
# Print shapes for other auxiliary inputs

print("Number of samples in X_train_img:", len(X_train_img))
print("Number of samples in X_train_auxiliary1:", len(X_train_auxiliary1))
# Print number of samples for other auxiliary inputs

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'],'b', label='training history')
pyplot.plot(history.history['val_loss'],'r',label='testing history')
pyplot.title("Train and Test Loss function")
pyplot.legend()
pyplot.show()

In [ ]:
loss = model.evaluate([X_test_img,X_test_auxiliary],y_test)
#accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

y_pred1 = model.predict([X_test_img,X_test_auxiliary])
y_pred = y_pred1.reshape(-1)
MAE=mae(y_test,y_pred)
RMSE=mse(y_test,y_pred,squared=False)
MAPE=mape(y_test,y_pred)
R2=rSquared(y_test,y_pred)

print("Mean Absolute Error:", MAE)
print("Root Mean Squared Error:", RMSE)
print("Mean Absolute Precentage Error",MAPE)
print("Coefficient of determination ",R2)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting y_test values
plt.scatter(y_pred, y_test, c='blue', label='True Values')

max_value = max(max(y_test), max(y_pred))
min_value = min(min(y_test), min(y_pred))
plt.plot([min_value, max_value], [min_value, max_value], color='red',linestyle='dashed',label='Reference line')


plt.xlabel('Predicted PM2.5 values (ug/m3)', fontsize=15)
plt.ylabel('True PM2.5 values (ug/m3)', fontsize=15)
plt.title('Scatter plot of Predicted Vs True PM2.5 Concentrations (Accuracy with Week)', fontsize=16)
#plt.legend()
#plt.title("Scatter plot of Predicted Vs True PM2.5 Concentrations")
plt.show()


In [ ]:
from sklearn.inspection import permutation_importance
# Function to evaluate the model with auxiliary feature ablation
def evaluate_feature_ablation(model, X_test_img, X_test_auxiliary, y_test, feature_index, feature_mean):
    X_test_aux_modified = X_test_auxiliary.copy()
    X_test_aux_modified[:, feature_index] = feature_mean
    y_pred = model.predict([X_test_img, X_test_aux_modified]).reshape(-1)
    return rSquared(y_test, y_pred)

# Calculate baseline R²
baseline_r2 = rSquared(y_test, y_pred)
print("Baseline R²:", baseline_r2)

# Feature names
feature_names = ['GreenIndex', 'UrbanIndex', 'NearRoad', 'Season', 'Is_Week']

# Calculate mean for each feature
feature_means = np.mean(X_test_auxiliary, axis=0)

# Evaluate the model with each feature set to its mean
r2_scores = []
r2_differences = []
for i, feature_mean in enumerate(feature_means):
    r2 = evaluate_feature_ablation(model, X_test_img, X_test_auxiliary, y_test, i, feature_mean)
    r2_scores.append(r2)
    r2_differences.append(baseline_r2 - r2)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_names, r2_differences, color='skyblue')
plt.axvline(x=0, color='r', linestyle='--', label='Baseline R² Difference')
plt.xlabel('R² Score Difference')
plt.ylabel('Features')
plt.title('Feature Importance based on R² Score Difference')
plt.legend()
plt.show()

# Calculate permutation importance
perm_importance = permutation_importance(model, [X_test_img, X_test_auxiliary], y_test, n_repeats=10, random_state=42)

# Get feature names
feature_names = ['GreenIndex', 'UrbanIndex', 'NearRoad', 'Season', 'Is_Week']

# Mean importances
mean_importance = perm_importance.importances_mean

# Error bars
importance_std = perm_importance.importances_std

# Plotting permutation importance
plt.figure(figsize=(10, 6))
plt.barh(feature_names, mean_importance, xerr=importance_std, color='skyblue')
plt.xlabel('Importance (change in R²)')
plt.ylabel('Features')
plt.title('Permutation Importance based on R²')

In [ ]:
import matplotlib.pyplot as plt

# Model names
models = ['LSTM', 'CNN', 'Hybrid']

# Accuracies
accuracies = [65, 60, 74]

# Create the bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(models, accuracies, color=['blue', 'green', 'purple'])

# Adding the text labels on the bars
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 5,
             f'{accuracy}%', ha='center', color='white', fontsize=12, fontweight='bold')

# Adding labels and title
plt.xlabel('Model', fontsize=15)
plt.ylabel('Accuracy (%)', fontsize=15)
plt.title('Model Accuracies Comparison', fontsize=18)
plt.ylim(0, 100)  # Set the y-axis limit to 100%

# Show the plot
plt.show()
